In [1]:
from __future__ import division

In [2]:
import tensorflow as tf
import tflearn

hdf5 not supported (please install/reinstall h5py)


In [3]:
import numpy as np

In [4]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
data_size = mnist.train.num_examples

In [7]:
from tqdm import tqdm

In [8]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
# import BayesianNN

In [10]:
# bnn = BayesianNN.BayesianFC([4,32,32,1])

In [11]:
import Bayesian_Conv_Layer, Bayesian_FC_Layer, Bayesian_Net

In [12]:
def bbnet(i, o, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_FC_Layer.Bayesian_FC(i, o, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [13]:
def bbnet_conv(i, f, stride, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_Conv_Layer.Bayesian_Conv(i, f, filter_stride=stride, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [14]:
std_init = 0.1
mean_init = 0.0
prior_mean = 0.0
prior_std = 1.0
l1 = bbnet_conv(1, 8, 2, mean_init, std_init, prior_mean, prior_std)
l2 = bbnet_conv(8, 8, 2, mean_init, std_init, prior_mean, prior_std)
l3 = bbnet(288, 10, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.softmax)

def bnet_model(input_tensor, local_reparam_trick=False):
    input_tensor = tf.reshape(input_tensor, shape=[-1,28,28,1])
    
    net = l1.sample(input_tensor, local_reparam_trick)
    net = l2.sample(net, local_reparam_trick)
    
    size = tf.reduce_prod(tf.shape(net)[1:])
    # Flatten
    net = tf.reshape(net, shape=[-1, size])
    
    net_out = l3.sample(net, local_reparam_trick)
    
    return net_out
# l1 = Bayesian_FC_Layer.Bayesian_FC(4, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l2 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l3 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l4 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l5 = Bayesian_FC_Layer.Bayesian_FC(32, 1, activation=tf.identity, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)

In [15]:
def bnet_prob(pred, target):
    return pred * target

In [16]:
bnet = Bayesian_Net.Bayesian_Net([l1, l2, l3], bnet_prob)

In [17]:
batch_size = 128

In [18]:
epochs = 5000

In [19]:
bnn_data_input = tf.placeholder(tf.float32, shape=[None, 28*28*1])
bnn_data_target = tf.placeholder(tf.float32, shape=[None, 10])
kls = tf.placeholder(tf.float32, shape=[])

bnet_loss, klloss, dataloss = bnet.loss(bnn_data_input, bnet_model, bnn_data_target,kl_scaling=kls, N=8)
bnet_output = bnet_model(bnn_data_input, local_reparam_trick=False)

copy_old = bnet.copy_variational_parameters()
kl_div = bnet.kl_new_and_old()

correct_preds = tf.equal(tf.argmax(bnet_output, 1), tf.argmax(bnn_data_target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

grads_to_apply = optim.compute_gradients(bnet_loss)
# print(grads_to_apply)

clipped_policy_grads = []
for (grad, var) in grads_to_apply:
    if grad is not None:
        clipped_policy_grads.append((tf.clip_by_norm(grad, 10), var))

# clipped_policy_grads = grads_to_apply
grad_op = optim.apply_gradients(clipped_policy_grads)

sess.run(tf.global_variables_initializer())

In [20]:
bnn_lr = 0.001

In [21]:
epoch_loss=[]
kl_loss=[]
data_loss=[]
kl_divs=[]
accs=[]

In [22]:
for e in tqdm(range(epochs)):
    
    x, y = mnist.train.next_batch(batch_size)
    
    kl = batch_size/data_size  
    
    feed_dict={kls:kl, bnn_data_input: x, bnn_data_target:y, lr:bnn_lr}
    
    _, l, kloss, dloss = sess.run([grad_op, bnet_loss, klloss, dataloss], feed_dict=feed_dict)

    epoch_loss.append(l)
    
    if e % 50 == 0 and e > 0:
        div = sess.run(kl_div)
        sess.run(copy_old)
        kl_divs.append(div)
    
    if e % 5 == 0:
        epoch_loss.append(l) 
        kl_loss.append(kloss)
        data_loss.append(dloss)
        
    if e % 10 == 0:
        test_dict = {bnn_data_input: mnist.test.images, bnn_data_target: mnist.test.labels}
        acc = sess.run([accuracy], feed_dict=test_dict)
        accs.append(acc)

100%|██████████| 5000/5000 [07:48<00:00, 10.68it/s]


In [23]:
loss_p = figure(width=500, height=500)

# Loss
# loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

loss_p.line(range(len(kl_loss)), kl_loss, line_width=0.5, color="orange")

loss_p.line(range(len(data_loss)), data_loss, line_width=0.5, color="purple")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="green")

show(loss_p)

In [24]:
loss_kl = figure(width=500, height=500)

loss_kl.line(range(len(kl_divs)), kl_divs, line_width=0.5, color="red")

show(loss_kl)

In [25]:
acc_g = figure(width=500, height=500)

acc_g.line(range(len(accs)), accs, line_width=0.5, color="blue")

show(acc_g)